In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable, grad
import numpy as np
from collections import OrderedDict
from torch.autograd import functional as F 

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cpu")

In [3]:
class DNN(nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        self.depth = len(layers) - 1
        self.activation = nn.Tanh
        
        layer_list = list()
        for i in range(self.depth -1):
            layer_list.append(
                (f'layer_{i}', nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append((f'activation_{i}', self.activation()))
        
        layer_list.append(
            (f'layer_{self.depth - 1}', nn.Linear(layers[-2], layers[-1]))
        )
        
        layer_dict = OrderedDict(layer_list)
        
        self.layers = nn.Sequential(layer_dict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [4]:
def true_solution(x):
    return -(x ** 4) / 24 + x ** 3 / 6 - x ** 2 / 4

In [17]:
class pinn():
    def __init__(self, x, layers):
        self.mse_cost_function = nn.MSELoss() 
        self.layers = layers
        self.x = torch.tensor(x, requires_grad=True).float().to(device)
        
        # deep neural networks
        self.dnn = DNN(layers).to(device)
        
        # optimizers: using the same settings
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"
        )

        self.iter = 0
        
    def net_y(self, x):  
        y = self.dnn(x)
        return y
    
    def net_f(self, x):
        """ The pytorch autograd version of calculating residual """
        y = self.net_y(x)
        
        y_x = torch.autograd.grad(
            y, x, 
            grad_outputs=torch.ones_like(y),
            retain_graph=True,
            create_graph=True
        )[0]
        
        y_xx= torch.autograd.grad(
            y_x, x, 
            grad_outputs=torch.ones_like(y_x),
            retain_graph=True,
            create_graph=True
        )[0]
        
        y_xxx = torch.autograd.grad(
            y_xx, x, 
            grad_outputs=torch.ones_like(y_xx),
            retain_graph=True,
            create_graph=True
        )[0]
        
        y_xxxx = torch.autograd.grad(
            y_xxx, x, 
            grad_outputs=torch.ones_like(y_xxx),
            retain_graph=True,
            create_graph=True
        )[0]
        
        f = y_xxxx + 1
        return f
    
    def loss_func(self):
        self.optimizer.zero_grad()
        
        y_pred = self.net_y(self.x)
        f_pred = self.net_f(self.x)
        loss_u = torch.mean((self.x - y_pred) ** 2)
        loss_f = torch.mean(f_pred ** 2)
        
        loss = loss_u + loss_f
        
        loss.backward()
        self.iter += 1
        if self.iter % 100 == 0:
            print(
                'Iter %d, Loss: %.5e, Loss_u: %.5e, Loss_f: %.5e' % (self.iter, loss.item(), loss_u.item(), loss_f.item())
            )
        return loss
    
    def train(self):
        self.dnn.train()
                
        # Backward and optimize
        self.optimizer.step(self.loss_func)

            
    def predict(self, X):
        x = torch.tensor(X, requires_grad=True).float().to(device)

        self.dnn.eval()
        y = self.net_y(x)
        f = self.net_f(x)
        y = y.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        return u, f

In [18]:
x_bc = np.random.uniform(low=0.0, high=1.0, size=(500,1))
y_bc = true_solution(x_bc)

In [19]:
layers = [1] + [20] * 5 + [1]

In [20]:
model = pinn(x_bc, layers)

In [21]:
model.train()

Iter 100, Loss: 2.39182e-04, Loss_u: 2.19404e-04, Loss_f: 1.97780e-05
Iter 200, Loss: 5.27211e-05, Loss_u: 1.83213e-05, Loss_f: 3.43998e-05
Iter 300, Loss: 3.98491e-06, Loss_u: 2.84307e-07, Loss_f: 3.70060e-06
Iter 400, Loss: 2.41905e-06, Loss_u: 1.12116e-06, Loss_f: 1.29790e-06
Iter 500, Loss: 2.04390e-06, Loss_u: 1.26961e-06, Loss_f: 7.74282e-07


In [22]:
model.predict(0.2)

RuntimeError: both arguments to matmul need to be at least 1D, but they are 0D and 2D